<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hard-coded-agents-championship" data-toc-modified-id="Hard-coded-agents-championship-1">Hard coded agents championship</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1">Imports</a></span></li><li><span><a href="#Agents" data-toc-modified-id="Agents-1.2">Agents</a></span><ul class="toc-item"><li><span><a href="#Random" data-toc-modified-id="Random-1.2.1">Random</a></span></li><li><span><a href="#Greedy" data-toc-modified-id="Greedy-1.2.2">Greedy</a></span></li><li><span><a href="#Risk-averse-goose" data-toc-modified-id="Risk-averse-goose-1.2.3">Risk averse goose</a></span></li></ul></li><li><span><a href="#Championship" data-toc-modified-id="Championship-1.3">Championship</a></span><ul class="toc-item"><li><span><a href="#Code" data-toc-modified-id="Code-1.3.1">Code</a></span></li><li><span><a href="#Matches" data-toc-modified-id="Matches-1.3.2">Matches</a></span></li></ul></li></ul></li></ul></div>

# Hard coded agents championship

## Imports

In [1]:
import os
import kaggle_environments
from kaggle_environments import make
from kaggle_environments import evaluate
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


%config Completer.use_jedi = False

Loading environment football failed: No module named 'gfootball'


In [2]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (30, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Agents

### Random

### Greedy

### Risk averse goose

https://www.kaggle.com/ilialar/risk-averse-greedy-goose

This notebook proposes a simple greedy straightforward solution. At each step:
- Mark all cells that can be dangerous on the next step as obstacles
- Find the shortest route to the nearest food item
- Make one step toward this food
- Repeat

## Championship

### Code

In [4]:
def sample_agent_keys(agents):
    keys = list(agents.keys())
    while 1:
        samples = np.random.choice(keys, size=4)
        if len(np.unique(samples)) > 1:
            break
    return samples

In [10]:
def get_rank(evaluate_ret):
    rank = np.array(evaluate_ret)*-1
    rank = np.argsort(rank, axis=1)+1
    return rank

### Matches

In [17]:
agents = {
    'random': 'random',
    'greedy': 'greedy',
    'risk_averse_goose': '../forum/agents/risk_averse_goose.py',
    'goose_on_a_healthy_diet': '../forum/agents/goose_on_a_healthy_diet.py'
}

In [18]:
sampled_keys = sample_agent_keys()
sampled_keys = list(agents.keys())
print(sampled_keys)

['random', 'greedy', 'risk_averse_goose', 'goose_on_a_healthy_diet']


In [19]:
ret = evaluate("hungry_geese", agents=[agents[key] for key in sampled_keys], num_episodes=25)
rank = get_rank(ret)
rank

array([[3, 4, 2, 1],
       [3, 2, 4, 1],
       [4, 2, 3, 1],
       [4, 3, 2, 1],
       [4, 3, 2, 1],
       [3, 4, 2, 1],
       [3, 4, 2, 1],
       [3, 4, 2, 1],
       [4, 3, 2, 1],
       [3, 4, 2, 1],
       [4, 2, 3, 1],
       [3, 2, 4, 1],
       [4, 2, 3, 1],
       [3, 4, 2, 1],
       [4, 3, 2, 1],
       [3, 4, 2, 1],
       [3, 4, 2, 1],
       [4, 2, 3, 1],
       [3, 4, 2, 1],
       [3, 4, 2, 1],
       [3, 4, 2, 1],
       [4, 3, 2, 1],
       [4, 3, 2, 1],
       [3, 1, 2, 4],
       [4, 2, 3, 1]])

TODO: 
- study how added agents work
- compute elo ranking from the agents that play each match and the ranking of each match.
- Create a better random agent that does not change direction, and compare with default one